# Semantic Similarity Example

## Installations

In [ ]:
!pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use a real OpenAI key, so for now we'll set them to empty strings.

In [ ]:
import os
os.environ['DEBUG'] = "1"  # Set to "1" if you want to use debug mode.
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [ ]:
from typing import Dict, Tuple
from prompttools.harness import PromptTemplateExperimentationHarness
from prompttools.experiment import OpenAICompletionExperiment

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [ ]:
prompt_templates = ["Answer the following question: {{input}}", "Respond the following query: {{input}}"]
user_inputs = [{"input": "Who was the first president?"}, {"input": "Who was the first president of India?"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [ ]:
harness = PromptTemplateExperimentationHarness(OpenAICompletionExperiment,
                                               "text-davinci-003", 
                                               prompt_templates, 
                                               user_inputs,
                                               model_arguments={"temperature": 0.9})

We can then run the experiment to get results.

In [ ]:
harness.run()
harness.visualize()

You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [ ]:
harness.visualize(pivot=True)

## Evaluate the model response

To evaluate the results, we'll define an eval function. We can use semantic distance to check if the model's response is similar to our expected output.

In [ ]:
from prompttools.utils import similarity


EXPECTED = {"Who was the first president?": "George Washington",
            "Who was the first president of India?": "Rajendra Prasad"}

def extract_responses(output) -> str:
    return [choice["text"] for choice in output["choices"]]


def measure_similarity(
    input_pair: Tuple[str, Dict[str, str]], results: Dict, metadata: Dict
) -> float:
    """
    A simple test that checks semantic similarity between the user input
    and the model's text responses.
    """
    distances = [
        similarity.compute(EXPECTED[input_pair[1]["input"]], response)
        for response in extract_responses(results)
    ]
    return min(distances)
    


Finally, we can evaluate and visualize the results.

In [ ]:
harness.evaluate("similar_to_expected", measure_similarity)
harness.visualize()